In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType


In [0]:
from pyspark.sql.types import StructType

from pyspark.sql.functions import *

df = spark.read.json('/Volumes/jpmc/naval/raw/1.json')
# Function to flatten the DataFrame
def flatten_df(nested_df):

    flat_cols = [col for col in nested_df.columns if not isinstance(nested_df.schema[col].dataType, StructType)]
    nested_cols = [col for col in nested_df.columns if isinstance(nested_df.schema[col].dataType, StructType)]

    flat_df = nested_df.select(flat_cols + [col(nested_col + '.' + field.name).alias(nested_col + '_' + field.name)
                                            for nested_col in nested_cols
                                            for field in nested_df.schema[nested_col].dataType.fields])
    return flat_df

# Flatten the DataFrame
flattened_df = flatten_df(df)
flattened_df.na.drop()
# Show the flattened DataFrame
flattened_df.show()
flattened_df.write.mode("overwrite").saveAsTable("jpmc.guru.patients_data")
